In [11]:
import carto2gpd
import geopandas as gpd
import pandas as pd
import numpy as np
import hvplot.pandas
import datetime as dt
import matplotlib.pyplot as plt
from census import Census
from us import states
import os
from google.cloud import storage


In [ ]:
#Pull data from google cloud

In [2]:
#Pull data from OPA --> should be querying from BigQuery

carto_url = "https://phl.carto.com/api/v2/sql"

table_name = "opa_properties_public"

#filter for Zoned Residential
where = "assessment_date >= '2022-01-01' and assessment_date <= '2023-04-01'"
where = where + " and zoning LIKE 'R%'"

opaRaw = carto2gpd.get(carto_url, table_name, where=where)

In [3]:
#Get tracts/neighborhoods for geodata
tracts = gpd.read_file('https://opendata.arcgis.com/datasets/8bc0786524a4486bb3cf0f9862ad0fbf_0.geojson')
neighborhoods = gpd.read_file('https://raw.githubusercontent.com/azavea/geo-data/master/Neighborhoods_Philadelphia/Neighborhoods_Philadelphia.geojson')

In [4]:
#Spatially join tracts, neighborhoods, and properties

opa1 = opaRaw.sjoin(tracts,how="left")
opa1 = opa1.drop(["index_right"], axis=1)
opa1 = opa1.sjoin(neighborhoods, how="left")


In [5]:
pd.set_option('display.max_columns', None)
opa1.sort_values('taxable_land').head()

,geometry,cartodb_id_left,assessment_date,basements,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,census_tract,central_air,cross_reference,date_exterior_condition,depth,exempt_building,exempt_land,exterior_condition,fireplaces,frontage,fuel,garage_spaces,garage_type,general_construction,geographic_ward,homestead_exemption,house_extension,house_number,interior_condition,location,mailing_address_1,mailing_address_2,mailing_care_of,mailing_city_state,mailing_street,mailing_zip,market_value,market_value_date,number_of_bathrooms,number_of_bedrooms,number_of_rooms,number_stories,off_street_open,other_building,owner_1,owner_2,parcel_number,parcel_shape,quality_grade,recording_date,registry_number,sale_date,sale_price,separate_utilities,sewer,site_type,state_code,street_code,street_designation,street_direction,street_name,suffix,taxable_building,taxable_land,topography,total_area,total_livable_area,type_heater,unfinished,unit,utility,view_type,year_built,year_built_estimate,zip_code,zoning,pin,objectid,OBJECTID,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,LOGRECNO,index_right,name,listname,mapname,shape_leng,shape_area,cartodb_id_right,created_at,updated_at
0,POINT (-75.18934 39.93429),1,2022-04-27T00:00:00Z,None,SEC 27TH ST,None,SR,VACANT LAND RESIDE < ACRE,6,VACANT LAND,32,None,None,None,62.0,0.0,120900,None,NaN,16.0,None,NaN,None,None,36,0,None,2642,None,2642 DICKINSON ST,OF PHILADELPHIA,16TH FLOOR,REDEVELOPMENT AUTHORITY,PHILADELPHIA PA,1234 MARKET ST,19107,120900,None,NaN,NaN,NaN,NaN,3460.0,None,REDEVELOPMENT AUTHORITY,OF PHILADELPHIA,364244805,E,None,2071-10-01T00:00:00Z,None,2071-10-01T00:00:00Z,1.0,None,None,None,PA,28480,ST,None,DICKINSON,None,0.0,0,F,992.0,NaN,None,None,None,None,I,None,None,19146,RSA5,1001179696,297397123,196,42,101,003200,42101003200,32,Census Tract 32,G5020,S,491790,0,+39.9333011,-075.1854187,10375,39,GRAYS_FERRY,Grays Ferry,Grays Ferry,25521.572356,3.131163e+07,7,2013-03-19 17:41:50.508000+00:00,2013-03-19 17:41:50.743000+00:00
28668,POINT (-75.08639 40.06363),300621,2022-04-27T00:00:00Z,None,"105'11"" N COTTMAN AVE",2171461,CA0,APTS 5-50 UNITS MASONRY,2,MULTI FAMILY,339,None,None,None,94.0,494410.0,211890,None,NaN,98.0,None,NaN,None,A,63,0,None,7312,None,7312 ROCKWELL AVE,2ND FLOOR,None,None,PHILADELPHIA PA,2042-48 ARCH ST,19103,706300,None,NaN,NaN,NaN,NaN,663.0,None,1260 HOUSING DEVELOPMENT,None,881223520,A,C,2010-01-28T00:00:00Z,147N190013,2010-01-28T00:00:00Z,1738177.0,None,None,None,PA,69060,AVE,None,ROCKWELL,None,0.0,0,F,9154.0,7622.0,None,None,None,None,I,1962,None,19111,RSA3,1001459527,297696333,123,42,101,033900,42101033900,339,Census Tract 339,G5020,S,1141156,0,+40.0685560,-075.0873418,10651,24,BURHOLME,Burholme,Burholme,18179.223341,1.045045e+07,34,2013-03-19 17:41:50.508000+00:00,2013-03-19 17:41:50.743000+00:00
28667,POINT (-75.03053 40.04380),300620,2022-04-27T00:00:00Z,None,"94'11 3/4"" E CRISPIN",2171461,CA0,APTS 5-50 UNITS MASONRY,2,MULTI FAMILY,331,None,None,None,150.0,1618400.0,693600,None,NaN,162.0,None,NaN,None,A,64,0,16,3408,None,3408-16 RHAWN ST,2ND FLOOR,None,None,PHILADELPHIA PA,2042-48 ARCH ST,19103,2312000,None,NaN,NaN,NaN,NaN,1310.0,None,1260 HOUSING DEVELOPMENT,None,881224700,A,B+,2010-01-28T00:00:00Z,116N230133,2010-01-28T00:00:00Z,1738177.0,None,None,None,PA,68180,ST,None,RHAWN,None,0.0,0,F,24331.0,25027.0,None,None,None,None,I,1968,None,19136,RM1,1001447449,297696332,166,42,101,033102,42101033102,331.02,Census Tract 331.02,G5020,S,718272,0,+40.0451050,-075.0272869,10642,74,HOLMESBURG,Holmesburg,Holmesburg,40376.277535,8.406447e+07,40,2013-03-19 17:41:50.508000+00:00,2013-03-19 17:41:50.743000+00:00
28666,POINT (-75.08691 40.07667),300619,2022-04-27T00:00:00Z,None,50' E BURHOLME AVE,2171461,CA0,APTS 5-50 UNITS MASONRY,2,MULTI FAMILY,341,None,None,None,152.0,336070.0,144030,None,NaN,49.0,None,NaN,None,A,63,0,6,304,None,304-06 LONEY ST,2ND FLOO

In [6]:
#Select columns

cols = ["market_value", #Num
        "sale_price", #Num
    "total_livable_area", #Num
    "total_area", #Num
    "garage_spaces", #Num
    "fireplaces", #Num
    "number_of_bathrooms", #Num
    "number_of_bedrooms",  #Num
    "number_stories",  #Num
    "central_air", #Cat
    "exterior_condition", #Cat
    "zip_code", #Cat
        "homestead_exemption", #Int --> #Cat
        "year_built", #Cat
        "zoning", #Cat
        "GEOID10", #Cat
        "name", #Cat
        "interior_condition",
        'geometry'
       ]

In [59]:
#Select only some columns
opa = opa1[cols].copy()

In [8]:
count = opa['market_value'].value_counts().get(0, 0)
length = len(opa)
div = round(count/length * 100,2)
print(f"Number of rows with zero value: {count} out of {length} values ({div}%)")

Number of rows with zero value: 35 out of 54515 values (0.06%)


In [9]:
max_m = opa['market_value'].max()
mean_m = opa['market_value'].mean()
median_m = opa['market_value'].median()

print(f"Max market value: {max_m}")
print(f"Mean: {mean_m}")
print(f"Median: {median_m}")

Max market value: 276892000
Mean: 450251.8302118683
Median: 229500.0


In [60]:
#Fix some categorical variables
opa["homestead_exemption"] = np.where(opa["homestead_exemption"] > 0, "T", "F")
opa["central_air"] = np.where((opa["central_air"] == "0") | (opa["central_air"] == "1"),
                               "F", "T")
opa["year_built"] = pd.to_numeric(opa["year_built"])
opa = opa.dropna(subset=['market_value', 
                         'geometry', 
                         "total_area",
                           "number_of_bathrooms",
                           "number_of_bedrooms",
                           "number_stories",
                           "year_built"])

Preparing the baseline model

In [19]:
# Train data for baseline model
## Load sklearn dependencies

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Model selection
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

# Pipelines
from sklearn.pipeline import make_pipeline

# Preprocessing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [20]:
# Split the data 70/30
train_set, test_set = train_test_split(opa, 
                                       test_size=0.3, 
                                       random_state=42)

# the target labels: log of sale price
y_train = np.log(train_set["market_value"])
y_test = np.log(test_set["market_value"])

# The features
feature_cols = [
    "total_area",
    "total_livable_area",
    "number_of_bathrooms",
    "number_of_bedrooms",
    "number_stories",
    "year_built"
]
X_train = train_set[feature_cols].values
X_test = test_set[feature_cols].values

In [21]:
# Make a linear model pipeline
linear_pipeline = make_pipeline(StandardScaler(), LinearRegression())

# Fit on the training data
linear_pipeline.fit(X_train, y_train)

# What's the test score?
linear_pipeline.score(X_test, y_test)

0.3282786559851061

In [22]:
# Make a random forest pipeline
forest_pipeline = make_pipeline(
    StandardScaler(), RandomForestRegressor(n_estimators=100, random_state=42)
)

# Run the 10-fold cross validation
scores = cross_val_score(
    forest_pipeline,
    X_train,
    y_train,
    cv=10,
)

# Report
print("R^2 scores = ", scores)
print("Scores mean = ", scores.mean())
print("Score std dev = ", scores.std())

R^2 scores =  [0.63029349 0.65911679 0.65421025 0.65584079 0.66106154 0.63665536
 0.62554548 0.65240768 0.63573784 0.62609118]
Scores mean =  0.6436960412480486
Score std dev =  0.013435783837437041


In [23]:
# Fit on the training data
forest_pipeline.fit(X_train, y_train)

# What's the test score?
forest_pipeline.score(X_test, y_test)

0.6431626705087692

In [24]:
#Add in categorical

num_cols = [
    "total_livable_area",
    "total_area",
    "number_of_bathrooms",
    "number_of_bedrooms",
    "number_stories",
    "year_built",
]

#Categorical columns
cat_cols = ["central_air", 
    "exterior_condition", 
    "interior_condition", 
    "homestead_exemption", 
    "zoning", 
    "name" 
           ]

transformer = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

In [25]:
# Split the data 70/30
train_set, test_set = train_test_split(opa, 
                                       test_size=0.3, 
                                       random_state=42)

# the target labels: log of sale price
y_train = np.log(train_set["market_value"])
y_test = np.log(test_set["market_value"])

pipe = make_pipeline(
    transformer, RandomForestRegressor(n_estimators=500, 
                                       random_state=42))


# Fit the training set
pipe.fit(train_set, y_train)

# What's the test score?
pipe.score(test_set, y_test)



0.8770096051365989

In [26]:
# Continuing with categorical variables...

ohe = transformer.named_transformers_['cat']
ohe_cols = ohe.get_feature_names_out()
features = num_cols + list(ohe_cols)

forest_model = pipe["randomforestregressor"]
importance = pd.DataFrame(
    {"Feature": features, "Importance": forest_model.feature_importances_}).sort_values("Importance")

In [27]:
# Graph feature importance

importance = importance.sort_values("Importance", ascending=False).iloc[:30]

importance.hvplot.barh(x="Feature", y="Importance")

:Bars   [Feature]   (Importance)

In [61]:
# Identify sale prices of nearest neighbors

from sklearn.neighbors import NearestNeighbors

def get_xy_from_geometry(df):
    """
    Return a numpy array with two columns, where the 
    first holds the `x` geometry coordinate and the second 
    column holds the `y` geometry coordinate
    
    Note: this works with both Point() and Polygon() objects.
    """
    # NEW: use the centroid.x and centroid.y to support Polygon() and Point() geometries 
    x = df.geometry.centroid.x
    y = df.geometry.centroid.y
    
    return np.column_stack((x, y)) # stack as columns

# Convert to meters and EPSG=3857
opa_3857 = opa.to_crs(epsg=3857)

# Extract x/y for sales
opaXY = get_xy_from_geometry(opa_3857)

N = 5
k = N + 1
nbrs = NearestNeighbors(n_neighbors=k)
nbrs.fit(opaXY)

opaDists, opaIndices = nbrs.kneighbors(opaXY)

# Handle any locations that have 0 distances
opaDists[opaDists == 0] = 1e-5 

opa["logOPADists"] = np.log10(opaDists[:, 1:].mean(axis=1))

total_opa = opa['sale_price'].values

neighboring_opa = total_opa[opaIndices[:, 1:]]

opa['laggedOPA'] = neighboring_opa.mean(axis=1)

In [66]:
#Get Census data

# Set the API
c = Census(os.environ.get("CENSUS_API_KEY"))

# Set the fields
fields = ("NAME", 'B19013_001E', 'B03002_001E')

# Get the Philadelphia County Census data
pa_census = c.acs5.state_county_tract(fields, state_fips="42", county_fips="101", tract="*", year=2020)
pa_census_df = pd.DataFrame(pa_census)
pa_df = pd.DataFrame(pa_census)
pa_df 

pa_df["GEOID"] = pa_df["state"] + pa_df["county"] + pa_df["tract"]

# Merge the census data
opa = opa.merge(pa_df, left_on="GEOID10", right_on="GEOID", how="left")

In [70]:
opa.rename(columns={'B19013_001E': 'median_income'}, inplace=True)

#remove columns with NaN values
opa = opa.dropna()

In [72]:
# Add in the lagged OPA and Census data to model

#Add in categorical

num_cols = [
    "total_livable_area",
    "total_area",
    "number_of_bathrooms",
    "number_of_bedrooms",
    "number_stories",
    "year_built",
    "laggedOPA",
    "median_income"]

#Categorical columns
cat_cols = ["central_air", 
    "exterior_condition", 
    "interior_condition", 
    "homestead_exemption", 
    "zoning", 
    "name" 
           ]

transformer = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

# Split the data 70/30
train_set, test_set = train_test_split(opa, 
                                       test_size=0.3, 
                                       random_state=42)

# the target labels: log of sale price
y_train = np.log(train_set["market_value"])
y_test = np.log(test_set["market_value"])

pipe = make_pipeline(
    transformer, RandomForestRegressor(n_estimators=500, 
                                       random_state=42))


# Fit the training set
pipe.fit(train_set, y_train)

# What's the test score?
pipe.score(test_set, y_test)

In [ ]:
def plot_feature_importances(pipeline, num_cols, transformer, top=20, **kwargs):
    """
    Utility function to plot the feature importances from the input
    random forest regressor.

    Parameters
    ----------
    pipeline :
        the pipeline object
    num_cols :
        list of the numerical columns
    transformer :
        the transformer preprocessing step
    top : optional
        the number of importances to plot
    **kwargs : optional
        extra keywords passed to the hvplot function
    """
    # The one-hot step
    ohe = transformer.named_transformers_["cat"]

    # One column for each category type!
    ohe_cols = ohe.get_feature_names_out()

    # Full list of columns is numerical + one-hot
    features = num_cols + list(ohe_cols)

    # The regressor
    regressor = pipeline["randomforestregressor"]

    # Create the dataframe with importances
    importance = pd.DataFrame(
        {"Feature": features, "Importance": regressor.feature_importances_}
    )

    # Sort importance in descending order and get the top
    importance = importance.sort_values("Importance", ascending=False).iloc[:top]

    # Plot
    return importance.hvplot.barh(
        x="Feature", y="Importance", flip_yaxis=True, **kwargs
    )

In [ ]:
plot_feature_importances(pipe, num_cols, transformer, top=30, height=500)